0. Pré-requisitos

In [3]:
!pip install selenium
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

root_path = "/content/drive/My Drive/Colab Notebooks/data"
img_path = os.path.join(root_path, "img")
for dir in [root_path, img_path]:
  os.mkdir(dir) if not os.path.exists(dir) else None

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
Mounted at /content/drive


1. Extraia informação de uma página web estática à sua escolha (https://www.meteoestrela.pt/previsoes/).

In [5]:
import requests
import json
import pandas
from bs4 import BeautifulSoup
from IPython.display import HTML

base_url = "https://www.meteoestrela.pt/previsoes/"

def download_image(url, id):
  try:
    filename = url.split("/")[-1]
    if "?" in filename:
      filename = filename.split("?")[0]
    filename = id + "." + filename.split(".")[-1]
    img_response = requests.get(url)
    if img_response.status_code < 200 or img_response.status_code > 299:
      raise Exception
    with open(os.path.join(img_path, filename), "wb") as image_file: # Open a new file in binary mode
      image_file.write(img_response.content)
    return filename
  except:
    print("Error downloading: " + url)
    return None

def extract(article, id):
  image_url = json.loads(article.select(".blog-image")[0].get("data-mk-image-src-set"))["default"]
  image = {
      "local": download_image(image_url, str(id)),
      "remote": image_url
  }
  title = article.select(".the-title > a")[0]
  return {
      "id": id,
      "date": article.select(".mk-blog-meta-wrapper > time")[0].get("datetime"),
      "title": title.text.strip(),
      "headline": article.select(".the-excerpt > p")[0].text.strip(),
      "url": title.get("href"),
      "image": image
  }

response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')
news = []
id = 1
for article in soup.select('.blog-grid-holder'):
  news.append(extract(article, id))
  id += 1
with open(os.path.join(root_path, "meteoestrela.pt_previsoes.json"), "w") as json_file:
  json.dump(news, json_file)

df = pandas.DataFrame(news)
pandas.set_option('display.max_colwidth', None)
image_formatter = lambda i: f'<img src="{i["remote"]}" width="128px">'
display(HTML(df.to_html(escape=False, formatters=dict(image=image_formatter))))

,id,date,title,headline,url,image
0,1,2024-03-27,Novo agravamento do estado do tempo Quinta-feira!,Agravamento do estado do tempo na Quinta-feira com chuva pontualmente forte e persistente principalmente no Norte e Centro.,https://www.meteoestrela.pt/previsoes/2024/03/27/novo-agravamento-do-estado-do-tempo-quinta-feira/,
1,2,2024-03-23,Muito frio e mais um bom nevão na Serra da Estrela na próxima semana!,"Mudança drástica do estado do tempo na Segunda-feira com uma descida bastante acentuada das temperaturas, voltamos a ter temperaturas negativas nas montanhas do Norte e Centro. São esperados [...]",https://www.meteoestrela.pt/previsoes/2024/03/23/muito-frio-e-mais-um-bom-nevao-na-serra-da-estrela-na-proxima-semana/,
2,3,2024-03-13,Temperaturas amenas para os próximos dias!,Temperaturas amenas para os próximos dias com alguns chuviscos no Minho e Douro Litoral podendo chegar às regiões do Interior.,https://www.meteoestrela.pt/previsoes/2024/03/13/temperaturas-amenas-para-os-proximos-dias/,
3,4,2024-03-04,"Chuva forte, vento e neve na Quinta-feira! Esperado grande nevão na Serra da Estrela","O Inverno chegou tarde mas chegou, vamos ter um final semana tipicamente Invernal com muita chuva, vento e mais neve nas montanhas!",https://www.meteoestrela.pt/previsoes/2024/03/04/chuva-forte-vento-e-neve-na-quinta-feira/,
4,5,2024-02-28,Regresso da chuva e da neve para o fim de semana!,Continuamos com tempo frio normal para a época pois afinal estamos no Inverno. Vamos ter novamente instabilidade com o regresso da chuva e da neve às montanhas nos próximos dias.,https://www.meteoestrela.pt/previsoes/2024/02/28/regresso-da-chuva-e-da-neve-para-o-fim-de-semana/,
5,6,2024-02-26,Tempo frio durante a semana!,"Semana com tempo frio devido a uma massa de ar polar com as temperaturas a descerem para valores negativos em algumas localidades do Norte e Centro do País,",https://www.meteoestrela.pt/previsoes/2024/02/26/tempo-frio-durante-a-semana/,
6,7,2024-02-18,"Regresso do frio, chuva e alguma neve nas montanhas!","Como já tinha referido este mês de Fevereiro tem duas ""faces"", depois deste tempo Primaveril regressa o tempo mais frio com chuva e alguma neve nas montanhas, assim na Quinta-feira já são [...]",https://www.meteoestrela.pt/previsoes/2024/02/18/regresso-do-frio-chuva-e-alguma-neve-nas-montanhas/,
7,8,2024-02-13,Voltamos a temperaturas máximas que podem chegar aos 22º/23ºc! Chuva na Quinta-feira.,"Depois deste episódio Invernal com neve as temperaturas do ar voltam a subir para valores acima da média, assim voltamos a temperaturas máximas que podem chegar aos 22º/23ºc j",https://www.meteoestrela.pt/previsoes/2024/02/13/voltamos-a-temperaturas-maximas-que-podem-chegar-aos-22o-23oc-chuva-na-quinta-feira/,
8,9,2024-02-06,"Regresso do Inverno com chuva, vento forte e neve!","Como já tinha sido adiantado a alguns dias vamos ter uma mudança radical do estado do tempo com o regresso do ""Inverno"". Assim vamos ter muita chuva, vento e neve nas montanhas nos próximos dias!",https://www.meteoestrela.pt/previsoes/2024/02/06/regresso-do-inverno-com-chuva-vento-forte-e-neve/,
9,10,2024-01-30,Continuação tempo seco com descida da temperatura. O Carnaval poderá ser “molhado”!,"Esta situação de bloqueio tudo indica que vai manter-se pelo menos até dia 7 de Fevereiro, depois disso tudo indica que o Anticiclone vá perdendo ""força"" e se vá deslocar mais para Sul [...]",https://www.meteoestrela.pt/previsoes/2024/01/30/continuacao-tempo-seco-com-descida-da-temperatura-o-carnaval-podera-ser-molhado/,
